# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,yellowknife,62.4560,-114.3525,18.29,20,20,6.17,CA,1715295603
1,1,hobyo,5.3505,48.5268,28.88,81,92,5.02,SO,1715295825
2,2,edinburgh of the seven seas,-37.0676,-12.3116,12.26,54,87,2.17,SH,1715295826
3,3,yatou,37.1500,122.3833,15.64,74,46,7.04,CN,1715295826
4,4,amga,60.8953,131.9608,1.53,50,33,3.01,RU,1715295827


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
#%%capture --no-display

# Configure the map plot
humidityPlot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["City", "Humidity"]
)

# Display the map
humidityPlot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values

#temperature is between 85 and 75 

idealCityDF = city_data_df.loc[
    (city_data_df["Max Temp"]>21) & (city_data_df["Max Temp"]<27)\
    &(city_data_df["Wind Speed"]<4.5)\
    &(city_data_df["Cloudiness"]==0)
]



# Drop any rows with null values
idealCityDF = idealCityDF.dropna()

# Display sample data
print(len(idealCityDF))
idealCityDF.head()

5


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
32,32,ardmore,34.1743,-97.1436,22.92,57,0,3.09,US,1715295840
235,235,belmonte,-15.8631,-38.8828,25.38,83,0,1.74,BR,1715295711
389,389,goldsboro,35.3849,-77.9928,25.50,69,0,4.12,US,1715295733
505,505,lewiston orchards,46.3804,-116.9754,25.95,36,0,4.12,US,1715296050
552,552,hotan,37.0997,79.9269,22.59,13,0,3.18,CN,1715296069


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = idealCityDF[["City","Country","Lat","Lng","Humidity"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
32,ardmore,US,34.1743,-97.1436,57,
235,belmonte,BR,-15.8631,-38.8828,83,
389,goldsboro,US,35.3849,-77.9928,69,
505,lewiston orchards,US,46.3804,-116.9754,36,
552,hotan,CN,37.0997,79.9269,13,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
   
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
   

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
   
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ardmore - nearest hotel: Best Western Plus
belmonte - nearest hotel: No hotel found
goldsboro - nearest hotel: Economy Inn
lewiston orchards - nearest hotel: Hampton Inn Lewiston
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)


,City,Country,Lat,Lng,Humidity,Hotel Name
32,ardmore,US,34.1743,-97.1436,57,Best Western Plus
235,belmonte,BR,-15.8631,-38.8828,83,No hotel found
389,goldsboro,US,35.3849,-77.9928,69,Economy Inn
505,lewiston orchards,US,46.3804,-116.9754,36,Hampton Inn Lewiston
552,hotan,CN,37.0997,79.9269,13,Hua Yi International Hotel (accepts foreigners)


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
32,ardmore,US,34.1743,-97.1436,57,Best Western Plus
235,belmonte,BR,-15.8631,-38.8828,83,No hotel found
389,goldsboro,US,35.3849,-77.9928,69,Economy Inn
505,lewiston orchards,US,46.3804,-116.9754,36,Hampton Inn Lewiston
552,hotan,CN,37.0997,79.9269,13,Hua Yi International Hotel (accepts foreigners)


In [18]:
#%%capture --no-display

# Configure the map plot
hotelPlot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 896,
    frame_height = 504,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    hover_cols=["City","Hotel Name","Humidity", "Country"]
)

# Display the map
hotelPlot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)